# PROBLEM STATEMENT

**"Implementing pre-trained ResNet-50"**

### Importing the Modules

In [26]:
from keras.applications import ResNet50

import numpy as np, pandas as pd, matplotlib.pyplot as plt, random

from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.resnet import preprocess_input

from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

### Loading the Cifar10 Dataset

In [27]:
from keras.datasets import cifar10

(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [28]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3)

print(X_train.shape)
print(X_val.shape)

print(y_train.shape)
print(y_val.shape)

(35000, 32, 32, 3)
(15000, 32, 32, 3)
(35000, 1)
(15000, 1)


### Pre-Processing of the Dataset

In [29]:
X_train = preprocess_input(X_train)
X_test = preprocess_input(X_test)

### Encoding of the Dataset

In [30]:
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test = to_categorical(y_test)

### Loading the pre-built model

In [31]:
resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

Freezing the pre-built layers

In [32]:
for layer in resnet_model.layers:
    layer.trainable = False

In [33]:
resnet_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 32, 32, 3)]          0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 38, 38, 3)            0         ['input_2[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 16, 16, 64)           9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 16, 16, 64)           256       ['conv1_conv[0][0]']          
 on)                                                                                       

### Defining our New Architecture

In [34]:
from keras.layers import Dense, Flatten
from keras.models import Sequential

model = Sequential()

model.add(resnet_model)

model.add(Flatten())

model.add(Dense(512, activation='relu'))

model.add(Dense(10, activation='softmax'))  # Output layer with 10 classes for CIFAR-10

### Model Building and Training

In [35]:
from keras.optimizers import Adam

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [36]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 1, 1, 2048)        23587712  
                                                                 
 flatten_1 (Flatten)         (None, 2048)              0         
                                                                 
 dense_2 (Dense)             (None, 512)               1049088   
                                                                 
 dense_3 (Dense)             (None, 10)                5130      
                                                                 
Total params: 24641930 (94.00 MB)
Trainable params: 1054218 (4.02 MB)
Non-trainable params: 23587712 (89.98 MB)
_________________________________________________________________


In [37]:
model.fit(X_train, y_train, batch_size=64, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
547/547 [==============================] - 15s 20ms/step - loss: 1.2468 - accuracy: 0.5987 - val_loss: 1.0233 - val_accuracy: 0.6341
Epoch 2/10
547/547 [==============================] - 10s 18ms/step - loss: 0.8532 - accuracy: 0.7004 - val_loss: 1.0249 - val_accuracy: 0.6456
Epoch 3/10
547/547 [==============================] - 9s 17ms/step - loss: 0.7006 - accuracy: 0.7500 - val_loss: 1.0544 - val_accuracy: 0.6449
Epoch 4/10
547/547 [==============================] - 10s 18ms/step - loss: 0.5630 - accuracy: 0.7996 - val_loss: 1.1397 - val_accuracy: 0.6450
Epoch 5/10
547/547 [==============================] - 9s 17ms/step - loss: 0.4361 - accuracy: 0.8457 - val_loss: 1.2290 - val_accuracy: 0.6382
Epoch 6/10
547/547 [==============================] - 9s 16ms/step - loss: 0.3195 - accuracy: 0.8875 - val_loss: 1.4690 - val_accuracy: 0.6389
Epoch 7/10
547/547 [==============================] - 9s 16ms/step - loss: 0.2428 - accuracy: 0.9148 - val_loss: 1.5255 - val_accuracy: 0.6

### Accuracy Testing and Predictions

In [38]:
score = model.evaluate(X_test, y_test, verbose=0)

print('Test loss:', score[0])

print('Test accuracy:', score[1])

Test loss: 2.002345085144043
Test accuracy: 0.6331999897956848
